In [6]:

import random
import numpy as np
def select_subset(array, subset_size):
    subset_indices = random.sample(range(len(array)), subset_size)
    subset = [array[i] for i in subset_indices]
    return np.array(subset)
origial_array=np.load('data/temp/sampled_coordinates.npy')
print(origial_array.shape)
coord_subset = select_subset(origial_array, 2000)
print(coord_subset.shape)


(5671, 3)
(2000, 3)


In [17]:
from functions.c_optimization_problem import initialize_dictionary
import numpy as np

Weight_Space = np.load("data/X_W_arrays/W_space.npy")

print(Weight_Space.shape)

(640, 438)


In [18]:
Dw = initialize_dictionary(Weight_Space, 20)
print(Dw.shape)

In [19]:


def initialize_coefficients(D, all_ellements, a):
    #X: (150, 500), d: (150X25), a: soothing factor
    coefficients=np.zeros((D.shape[1] , all_ellements.shape[1]))
    
    for j, sample in enumerate(all_ellements.T):
        for i, word in enumerate(D.T):
            diff = np.linalg.norm(sample-word)**2
            c_1=np.exp(-a * diff)
            coefficients[i][j]=c_1
    coefficients[:,j]/=np.sum(coefficients[:,j])
    return coefficients

(640, 20)


In [20]:
from functions.c_optimization_problem import initialize_coefficients

Lw=initialize_coefficients(Dw, Weight_Space, 0.2)
print(Lw.shape)


In [22]:
np.save("data/dictionaries/Dw", Weight_Space)
np.save("data/dictionaries/Lw", Lw)

In [31]:
def load_roi(subj_no):
    roi = np.load("data/MRI_MASKS/roi_masks_dataset/roi_"+ subj_no + ".npy")
    return roi 

roi = load_roi("9002430")
print(np.unique(roi), roi.shape)


def gather_roi_indices(roi_array):
    roi_inices=np.array(np.where(roi_array == 1)).T
    # print(roi_inices)
    roi_perimeter= [np.array(coord) for coord in roi_inices]
    return np.array(roi_perimeter)
# roi_perimeter=gather_roi_indices(roi)
# print(roi_perimeter.shape, roi_perimeter)


[0 1] (384, 384, 160)
(20481580, 3)


In [28]:
roi_perimeter = []
for x in range(88,302,6):
    for y in range(63,321,6):
        for z in range(12,147,6):
            roi_perimeter.append([x,y,z])

roi_perimeter = np.array(roi_perimeter)

In [13]:
random_state=1
kmeans_model1=KMeans(n_clusters=120, verbose=False, init='k-means++', random_state=random_state)
kmeans_model1.fit(roi_perimeter)

c:\MySoftware\python3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=120, random_state=1, verbose=False)

In [14]:
kmeans_model2=KMeans(n_clusters=5, verbose=False, init='k-means++', random_state=random_state)
kmeans_model2.fit(kmeans_model1.cluster_centers_)

c:\MySoftware\python3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=5, random_state=1, verbose=False)

In [15]:
kmeans_model2.cluster_centers_

array([[143.21343437, 284.26397253,  78.97700227],
       [249.42306288, 130.42189711,  78.86156424],
       [141.64832759, 113.13232281,  76.02412297],
       [250.02234419, 257.040163  ,  79.61890683],
       [148.85776705, 202.2834326 ,  82.29225956]])

In [16]:
roi_subset = []
subregion_centers=kmeans_model2.cluster_centers_
for coordinate in roi_perimeter:
    x,y,z=coordinate
    if x%5==0 and y%5==0 and z%5==0:
        roi_subset.append(coordinate)
roi_subset=np.array(roi_subset)
print(roi_subset.shape)

(60372, 3)


In [17]:
first_label = kmeans_model1.predict(roi_subset)
final_label = kmeans_model2.predict(kmeans_model1.cluster_centers_[first_label])


In [18]:
subregion_1 = roi_subset[final_label==0]
subregion_2 = roi_subset[final_label==1]
subregion_3 = roi_subset[final_label==2]
subregion_4 = roi_subset[final_label==3]
subregion_5 = roi_subset[final_label==4]


In [19]:
print(subregion_1.shape, " -- ", subregion_2.shape, " -- ",subregion_3.shape, " -- ",subregion_4.shape, " -- ",subregion_5.shape)

(9561, 3)  --  (15142, 3)  --  (11851, 3)  --  (13755, 3)  --  (10063, 3)


In [21]:
np.save("coordinates/sub_1_coord.npy", subregion_1)
np.save("coordinates/sub_2_coord.npy", subregion_2)
np.save("coordinates/sub_3_coord.npy", subregion_3)
np.save("coordinates/sub_4_coord.npy", subregion_4)
np.save("coordinates/sub_5_coord.npy", subregion_5)

In [ ]:
coordinates= np.load("data/temp/general_coordinates.npy")
dir1 = os.listdir("data/MRI_MASKS/subjects")
dir2 = [(os.listdir("data/MRI_MASKS/segmentation_masks")[i]).split(".")[0] for i in range(len(os.listdir("data/MRI_MASKS/segmentation_masks")))]
dir3 = [(os.listdir("data/MRI_MASKS/roi_masks_dataset")[i]).split(".")[0][4:] for i in range(len(os.listdir("data/MRI_MASKS/roi_masks_dataset")))]
common_subjects = list(set(dir1).intersection(dir2).intersection(dir3))

In [63]:
X = np.load("data/X_W_arrays/X_space.npy")
W = np.load("data/X_W_arrays/W_space.npy")

print(X.shape, W.shape)

(125, 437) (640, 438)


In [65]:
import os
dir1 = os.listdir("data/MRI_MASKS/subjects")
dir2 = [(os.listdir("data/MRI_MASKS/segmentation_masks")[i]).split(".")[0] for i in range(len(os.listdir("data/MRI_MASKS/segmentation_masks")))]
common_subjects = list(set(dir1).intersection(dir2))
print(len(common_subjects))

437


In [66]:
dir1 = os.listdir("data/MRI_MASKS/subjects")
dir2 = [(os.listdir("data/MRI_MASKS/segmentation_masks")[i]).split(".")[0] for i in range(len(os.listdir("data/MRI_MASKS/segmentation_masks")))]
common_subjects = list(set(dir1).intersection(dir2))
print(len(common_subjects))

437


In [71]:

# print(np.load("data/dictionaries/Dw.npy").shape)
Dx = initialize_dictionary(X, 25)
print(Dx.shape)
np.save("data/dictionaries/Dx.npy",Dx)

(125, 25)


In [74]:
Lx = initialize_coefficients(Dx, X, 0.8) #(25, 437)

In [76]:
np.save("data/dictionaries/Lx.npy", Lx)

In [77]:
import numpy as np
from functions.c_optimization_problem import *

W = np.load("data/X_W_arrays/W_space.npy")
Dw = initialize_dictionary(W, 25)
Lw = initialize_coefficients(Dw, W, 0.8)

In [78]:
np.save("data/dictionaries/W.npy", W)
np.save("data/dictionaries/Dw.npy", Dw)
np.save("data/dictionaries/LW.npy", Lw)

In [12]:
import numpy as np 
roi1=np.load('data/MRI_MASKS/roi_masks_dataset/roi_9001104.npy')
roi2=np.load('data/MRI_MASKS/roi_masks_dataset/roi_9023193.npy')
roi3=np.load('data/MRI_MASKS/roi_masks_dataset/roi_9011420.npy')

coord=np.load('data/temp/sampled_coordinates.npy')

i=0

for coordinate in coord:
    x,y,z = coordinate
    if roi2[x,y,z] ==roi1 [x,y,z] == roi3 [x,y,z]:
        i+=1


35.07317933345089 %


# TESTING QCQP

In [9]:
def initialize_dictionary(all_elements, Dictionary_Size):
    
    k = Dictionary_Size
    descr_kmeans=KMeans(n_clusters=k, random_state=0)
    descr_kmeans.fit(all_elements.T)
    dictionary = descr_kmeans.cluster_centers_  

    return dictionary.T

#Soft voting to initialize Lx (Zhang et al 2013 - 2.Soft Assignment Coding)
def initialize_coefficients(D, all_ellements, a):
    #X: (150, 500), d: (150X25), a: soothing factor
    coefficients=np.zeros((D.shape[1] , all_ellements.shape[1]))
    
    for j, sample in enumerate(all_ellements.T):
        for i, word in enumerate(D.T):
            diff = np.linalg.norm(sample-word)**2 
            c_1=np.exp(-a * diff)
            coefficients[i][j]=c_1
    coefficients[:,j]/=np.sum(coefficients[:,j])
    return coefficients


In [3]:
import numpy as np
from scipy.optimize import minimize#, NonlinearConstraint 
from sklearn.cluster import KMeans 
# Objective function
def objective(dx, X, Lx):
    dx =dx.reshape(dx.shape[0],1)
    return np.linalg.norm(X - dx @ Lx, ord='fro')#**2

# Constraint function
def constraint(dx):
    return 1 - np.linalg.norm(dx, ord=2)

def constraint_jacobian(dx):
    return - (dx / np.linalg.norm(dx, ord=2))

X = np.random.rand(150, 500)
Dx_init = initialize_dictionary(X, 25)
Lx = initialize_coefficients(Dx_init, X, 0.8)


c:\MySoftware\python3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [4]:
n = X.shape[1]
        
        
        # Constraint definition
constraint_definition = {'type': 'ineq', 'fun': constraint}     

def col_by_col_step(Dx_init):

    Dx = Dx_init.copy()
    
    for i, dx_init in enumerate(Dx_init.T):
        # Bounds for dx
        dx_init = np.reshape(dx_init, [dx_init.shape[0], 1])
        # constraint_definition = NonlinearConstraint(constraint, -np.inf, 1, jac=constraint_jacobian)
        # objective =  cp.Minimize(cp.sum_squares( + Di@L[i,:].reshape(1,no_samples)))
        A = X - np.sum([Dx[:,j].reshape(Dx[:,j].shape[0],1)@Lx[j,:].reshape(1,n) for j in range(Dx.shape[1]) if j!=i], axis=0)
        B = Lx[i, :].reshape(1,n)
        result = minimize(objective, dx_init, args=(A, B), method= 'COBYLA', constraints=[constraint_definition], options={'maxiter' : 1000 , 'disp' : False })

        # Obtained solution
        dx_optimal = result.x
        
        # print(np.linalg.norm(dx_optimal))
        Dx[:,i] = dx_optimal

    return Dx

Dx = col_by_col_step(Dx_init)

C:\Users\ChrisIoannidis\AppData\Local\Temp\ipykernel_11496\2448636275.py:18: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective, dx_init, args=(A, B), method= 'COBYLA', constraints=[constraint_definition],jac = constraint_jacobian ,options={'maxiter' : 1000 , 'disp' : False })
c:\MySoftware\python3\Lib\site-packages\scipy\optimize\_minimize.py:549: RuntimeWarning: Method COBYLA does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,


In [5]:
def optimization_loop(Dx_init):
    norms =[]
    Dx = Dx_init.copy()
    for i in range(100): 
        Dx = col_by_col_step(Dx)
        print(np.linalg.norm(X - Dx @ Lx, ord='fro'))
    return Dx

print('before' , np.linalg.norm(X - Dx @ Lx, ord='fro'))


Dx = optimization_loop(Dx_init)


print('after' , np.linalg.norm(X - Dx @ Lx, ord='fro'))
        

before 157.93763276395114


C:\Users\ChrisIoannidis\AppData\Local\Temp\ipykernel_11496\2448636275.py:18: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective, dx_init, args=(A, B), method= 'COBYLA', constraints=[constraint_definition],jac = constraint_jacobian ,options={'maxiter' : 1000 , 'disp' : False })


157.93763276395114
157.95099537042518
157.98538950860853
157.982432517066
157.98440889237486
157.99306041789603
157.9745768613824
157.98632085643854
157.98073586379698
157.98531975626346


In [1]:
import cvxpy as cvx

print(cvx.__version__)




1.3.1


In [2]:
import cvxpy as cp

In [16]:

def update_dictionaries_step(X, D, L, n, no_samples):
    k = 25
    # (150x500) = (150x1)(1x500)
    for i in range(k):    
        Di = cp.Variable((n,1))
        # print("old: ",D[:10,i])
        objective =  cp.Minimize(cp.sum_squares(X - np.sum([D[:,j]@L[j,:].reshape(1,no_samples) 
                                                           for j in range(k) if i!=i]) + Di@L[i,:].reshape(1,no_samples)))
                                                 # cp.Minimize(np.linalg.norm(X - Dx@Lx, "fro"))
        
        constraints = [cp.norm2(Di) <= 1]

        prob = cp.Problem(objective, constraints) 
        # try:
        prob.solve(solver='OSQP',verbose=True)
        # except cp.error.SolverError:
        #     prob.solve(solver='SCS')

        print(prob.status) #infea/sible_inaccurate
        D[:,i] = Di.value.reshape(n,) #AttributeError: 'NoneType' object has no attribute 'reshape'

    return D


In [13]:
X = np.random.rand(150, 500)
Dx_init = initialize_dictionary(X, 25)
Lx = initialize_coefficients(Dx_init, X, 0.8)
n = 150
no_samples=500

c:\MySoftware\python3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


# LASSO

In [6]:
import numpy as np
from sklearn.cluster import KMeans 

W = np.random.uniform(0,1, [645, 500])
Dw = initialize_dictionary(W, 25)
Lx = np.random.uniform(0,1, [25, 500])
M = np.random.uniform(0,1, [25, 25])

c:\MySoftware\python3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [7]:
import numpy as np 
from sklearn.linear_model import LassoLars
 

def lars_col_by_col(X, y , alpha=0.01):
    lasso = LassoLars(alpha=alpha)
    lasso.fit(X, y)
    return lasso.coef_


def update_Lw_step(W, Dw, M, Lx, gamma=0.7, alpha=0.01):

    Y = np.vstack([W, gamma * M @ Lx])
    X = np.vstack([Dw, gamma * np.identity(25)])
    Lw = np.zeros([Dw.shape[1], W.shape[1]])
    
    for i, y in enumerate(Y.T):
    
        lw_col = lars_col_by_col(X, y)
        Lw[:, i] = lw_col

    return Lw

Lw = update_Lw_step(W, Dw, M, Lx, gamma=0.7, alpha=0.01)


In [23]:
import numpy as np
coordinate_cube = np.load('coordinates/coordinate_cube.npy')
print(coordinate_cube.shape)
# coordinates/coordinate_cube.npy

(870966, 3)


In [2]:
import SimpleITK as sitk
import os

In [18]:
def get_lists_of_of_paths(directory):
    file_list = os.listdir(directory)
    file_paths = []
    for file in file_list:
        file_path = os.path.join(directory, file)
        file_paths.append(file_path)

    return file_paths  

def svm_coordinate_sampling(segm, roi):
    coord_labels = []
    coordinates = []
    
    mask_values = segm[coordinate_cube[:,0], coordinate_cube[:,1], coordinate_cube[:,2]]

    threshold_4 = np.count_nonzero(mask_values == 4) #.shape[0]#len(segm[segm[coordinate_cube]==4])
    
    threshold_1=int(threshold_4/1.2)
    threshold_2=int(threshold_4*1.3)
    threshold_3=int(threshold_4/1.1)
    threshold_0=int(threshold_4/1.3)
   
    i_0,i_1,i_2,i_3,i_4 =0, 0, 0, 0, 0
    
    print(threshold_0,threshold_1,threshold_2,threshold_3,threshold_4)
        
        
    for coordinate in coordinate_cube:
        x,y,z = coordinate
        if roi[x,y,z]==1 and 360>x>6 and 360>y>6 and 150>z>6:
            if segm[x,y,z]==0 and i_0<threshold_0:
                coordinates.append(coordinate)
                i_0+=1
                coord_labels.append(0)
            elif segm[x,y,z]==1 and i_1<threshold_1:
                coordinates.append(coordinate)
                i_1+=1
                coord_labels.append(1)
            elif segm[x,y,z]==2 and i_2<threshold_2:
                coordinates.append(coordinate)
                i_2+=1
                coord_labels.append(2)
            elif segm[x,y,z]==3 and i_3<threshold_3:
                coordinates.append(coordinate)
                i_3+=1
                coord_labels.append(3)
            elif segm[x,y,z]==4 and i_4<threshold_4:
                coordinates.append(coordinate)
                i_4+=1 
                coord_labels.append(4)
                
    coord_labels=np.array(coord_labels)
    unique_values, counts = np.unique(coord_labels, return_counts=True)

    for value, count in zip(unique_values, counts):
        print(f"{value}: {count}")
    coordinates = np.array(coordinates)  
    np.random.shuffle(coordinates)      
    return coordinates


def fn_scan_to_array(base_path):
    mid_file=get_lists_of_of_paths(base_path)[0]
    slice_list=get_lists_of_of_paths(mid_file)
    for i in range(160):
        if i==0:    
            slice = sitk.ReadImage(get_lists_of_of_paths(mid_file)[0])
            scan_array3D = (sitk.GetArrayFromImage(slice)).reshape(384,384)
        else:  
            slice = sitk.ReadImage(get_lists_of_of_paths(mid_file)[i])
            slice_array= sitk.GetArrayFromImage(slice).reshape(384,384)
            scan_array3D=np.dstack((scan_array3D, slice_array))
    return scan_array3D  # (384, 384, 160)



def fn_segm_mask_to_array(subject_name):

    mhd_path = "data/" + subject_name +"/"+subject_name+".segmentation_masks.mhd"
    segm_mask = sitk.GetArrayFromImage(sitk.ReadImage(mhd_path, sitk.sitkFloat32))
    return np.flip(segm_mask, axis=0) # (384, 384, 160)


In [22]:
import os
import numpy as np
import SimpleITK as sitk
mri_scan = fn_scan_to_array('data/9004175')
segm_mask = fn_segm_mask_to_array('9004175')
roi = np.load('data/9004175/roi.npy')
coordinates = svm_coordinate_sampling(segm_mask, roi)

print(coordinates.shape)

1993 2160 3369 2356 2592
0: 1993
1: 2160
2: 3369
3: 2356
4: 2592
(12470, 3)


In [24]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Step 3: Prepare the data for SVM training
descriptors = []
labels = []
for coord in coordinates:
    descriptor = fn_hog_lbp_descriptor(coord, mri_scan)
    # if descriptor!= None # makeshiftSIFT(mri_scan, coord)
    descriptor = np.reshape(descriptor, [descriptor.shape[1],])
    descriptors.append(descriptor)
    value = segm_mask[coord[0], coord[1], coord[2]]
    if value == 0:
        labels.append(0)
    elif value == 1 or value == 3:
        labels.append(1)
    elif value == 2 or value == 4:
        labels.append(2)
    else:
        labels.append(value)   
    
    if len(descriptors) == 1:
        print(descriptor.shape)
    # labels.append(value)
    
    
X = np.array(descriptors)
y = np.array(labels)

print('created datasets')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape )
# Step 4: Train the SVM
svm = SVC(kernel= 'linear', decision_function_shape ='ovr')
print('fitting model')
svm.fit(X_train, y_train)

# Step 5: Predict on the test set
y_pred = svm.predict(X_test)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

(192,)
created datasets
(9976, 192) (9976,)
fitting model
Accuracy: 0.8364073777064955


In [34]:
coord = coordinates[1500]
descriptor = fn_hog_lbp_descriptor(coord, mri_scan)

value = segm_mask[coord[0], coord[1], coord[2]]

print("true", value)
print("pred_1", svm.predict(descriptor))
descriptor =descriptor.reshape(192,)
decision_value = np.dot(descriptor, svm.coef_.T ) + svm.intercept_
predicted_class = decision_value.argmax()

print("Predicted class:", predicted_class)

[[ 0.00000000e+00 -4.82404830e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  5.90571147e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -1.39435215e+00  0.00000000e+00  0.00000000e+00
   6.79204518e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   3.52002782e+00 -8.41468938e-01 -4.31640925e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  3.41044426e-06  0.00000000e+00
  -3.06098773e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -7.42952416e+00  6.10311588e-06 -9.37289719e-01 -5.35576030e+00
  -4.12607454e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   4.24206797e-05  0.00000000e+00  7.38822306e-01 -6.28921057e+00
  -2.00607068e+00 -2.36809004e+00 -2.32649049e-01 -4.53459392e-01
  -4.97446268e-01 -9.79710728e-01 -1.40538534e+00 -1.38608296e+00
  -1.52930100e+00 -1.11802621e+00 -1.84614010e+00 -1.03149456e+00
  -8.60188661e-01 -9.58027876e-01 -1.65182288e-01 -3.22590014e-01
   2.60798

In [8]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.72      0.74       670
           1       0.83      0.88      0.85       959
           2       0.84      0.82      0.83      1180

    accuracy                           0.82      2809
   macro avg       0.81      0.81      0.81      2809
weighted avg       0.82      0.82      0.82      2809



In [20]:
from utils import getAverageMask, structure_tensor_3D, getGradients
from trimesh.creation import icosphere
from sklearn.utils.extmath import cartesian
from sklearn.metrics.pairwise import linear_kernel
import math
import itertools
from sklearn.preprocessing import normalize
from scipy.stats import kurtosis
from scipy.special import sph_harm
from scipy.ndimage.interpolation import map_coordinates

C:\Users\ChrisIoannidis\AppData\Local\Temp\ipykernel_26764\3037991853.py:10: DeprecationWarning: Please use `map_coordinates` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import map_coordinates


In [21]:
def lbp_ri_sh(point, img, patch_size, sph_degree, ico_radius, ico_level, n_bins = 30, concatenate = True):
    '''
    Computes a 3D LBP texture descriptor for a region centered around a voxel. The intensity values 
    of the neighboring voxels is treated as a spherical function, and decomposed into a sum of
    spherical harmonics, achieving rotational invariance. A histogram of the texture codes is computed for
    each frequency (band) and the final descriptor is the concatenation of the above histograms.

    Reference : 3D LBP-Based Rotationally Invariant Region Description, 
                Banerjee J., Moelker A., Niessen W., Walsum  v. T., 
                ACCV 2012 Workshops, Part I, LNCS 7728, pp. 26-37, 2013

    ...

    Parameters
    ----------
    point : tuple
        the point to be described
    img : ndarray (width x height x depth)
        the MR image
    patch_size : int
        size of cellular patch around point
    sph_degree : int
        degree up to which to expand to spherical harmonics
    ico_radius : float
        radius of icosahedron to uniformly sample intensities around patch voxels
    ico_level : int
        controls refinement level of icosahedron
    n_bins : int
        number of bins for LBP histograms
    concatenate : bool
        if true, concatenate histograms, otherwise weighted aggregation

    Returns
    ------- 
    D : ndarray 
        LBP descriptor

    '''

    #extract image patch
    psize = patch_size // 2
    x, y, z = point
    patch = img[x - psize - int(math.ceil(ico_radius)) : x + psize + int(math.ceil(ico_radius)) + 1,
                y - psize - int(math.ceil(ico_radius)) : y + psize + int(math.ceil(ico_radius)) + 1,
                z - psize - int(math.ceil(ico_radius)) : z + psize + int(math.ceil(ico_radius)) + 1]
    
    patch_coords = cartesian((range(patch_size), range(patch_size), range(patch_size))) + 1    

    #construct icosahedron for uniform sampling on sphere surface
    ico = icosphere(subdivisions = ico_level, radius = ico_radius)
    ico_coords = np.array(ico.vertices)
    theta = np.arccos(ico_coords[:, 2] / ico_radius)
    phi = np.arctan2(ico_coords[:, 1], ico_coords[:, 0])

    #get Spherical Harmonics expansion coefficients (up to degree sph_degree)
    m = list(itertools.chain.from_iterable([[i for i in range(-n,n+1)] for n in range(sph_degree)]))
    m = np.array(m)

    l = list(itertools.chain.from_iterable([[k for _ in range(2 * k + 1)] for k in range(sph_degree)]))
    l = np.array(l)

    Y = sph_harm(m[None, :], l[None, :], theta[:, None], phi[:, None])

    #sample sphere neighbors for each voxel in patch and interpolate intensity
    mapped_coords = patch_coords[None, :, :] + ico_coords[:, None, :]
    mapped_int = map_coordinates(patch, mapped_coords.T, order = 3)
    center_int = patch[ico_radius : -ico_radius, ico_radius : -ico_radius, ico_radius : -ico_radius]

    #Compute kurtosis (for better rotation invariance)
    kurt = kurtosis(mapped_int)

    #Apply sign function and pass obtain spherical expansion coefficients for each sample
    f = np.greater_equal(center_int.ravel()[:, None], mapped_int).astype('int')
    c = f.dot(Y)

    #obtain frequency components of threshold function by integrating and normalizing over orders m
    f = np.multiply(c[:, None, l == 0], Y[None, :, l == 0])
    for n in range(1, sph_degree):
        f = np.concatenate((f, np.sum(np.multiply(c[:, None, l == n], Y[None, :, l == n]),
                            axis=2, keepdims=True)), axis=2)
    f = np.sqrt(np.sum(f**2, axis=1))

    #keep real parts of decomposition and kurtosis
    f = np.real(f)
    kurt = np.real(kurt)

    #extract histograms
    H = np.histogram(kurt, bins = n_bins)[0]
    for i in range(sph_degree):
        H = np.column_stack((H, np.histogram(f[:, i], bins = n_bins)[0]))
    H = normalize(H, axis = 0)

    #Return Descriptor (concatenated or aggregated histograms)
    if concatenate is True:
        D = H.T.ravel()
    else: 
        D = H.sum(axis = 1)
    D = D / np.linalg.norm(D)

    return D





def hog_3d_proj(point, image, psize = 5, rsize = 15, orientation = 'vertices', level = 1, mode = 'concatenated', 
                rot_inv = False, norm = 'l2'):
    '''
    Computes a 3D variant of the HOG Descriptor for an image region centered arounda voxel
    The Region of size (rsize x rsize x rsize) is compartmentalized into a set of disjoint patches,
    each of size (psize x psize x psize). A histogram of oriented gradients is computed for each patch, 
    with the orientation bins corresponding to vertices of centers of faces of a regular (refined) icosahedron.
    The final descriptor is a weighted aggregate of those histograms. Currently, implementation supports regions arranged in
    3x3x3 patches.

    Reference: Alexander Klaser, Marcin Marszalek, Cordelia Schmid. 
               A Spatio-Temporal Descriptor Based on 3D-Gradients. 
               BMVC 2008 - 19th British Machine Vision Conference, Sep 2008, Leeds, United Kingdom.pp.275:1-10. 
               DOI:10.5244/C.22.99

    ...

    Parameters
    ----------
    point : array - like
        the voxels to be characterized
    image : ndarray
        the image containing the voxels
    psize : int
        size of patches in region
    rsize : int
        size of region around central voxel
    orientation : string
        whether to associate histogram bins with vertices of centroids of faces of the icosahedron
    ico_coords : string
        coordinate system to define icosahedron on
    level : int
        number of refienement steps for icosahedron
    mode : string
        chooses whether to concatenate or aggregate patch histograms to form final descriptor
    
    Returns
    -------
    D : ndarray
        voxel descriptor

    '''

    #sanity check
    assert type(rsize // psize) == int, print("Wrong combination of regional and patch sizes")

    #set params
    rs = rsize // 2
    ps = psize // 2
    ncells = rsize // psize
    
    # get icosahedron
    ico = icosphere(subdivisions = level)
    if orientation  == 'faces':
        axes = np.array(ico.face_normals)
    else:
        axes = np.array(ico.vertices)

    # get average masks
    region_mask = getAverageMask(rsize // psize, 'manhattan')
    patch_mask = getAverageMask(psize, 'manhattan')

    #calculate partial derivatives
    x, y, z = point 
    xp = range(- rs + ps, rs - ps + 1, psize)
    yp = range(- rs + ps, rs - ps + 1, psize)
    zp = range(- rs + ps, rs - ps + 1, psize)
    patch_centers = cartesian((xp, yp, zp))
    patch_locations = patch_centers + psize

    # extracting +1 voxel in each direction for computational consistency
    region = image[x - rs - 1 : x + rs + 2,
                   y - rs - 1 : y + rs + 2,
                   z - rs - 1 : z + rs + 2]
        
    i_dx, i_dy, i_dz = getGradients(region)

    #get gradients at the patch level
    dx = np.array([i_dx[ploc[0] : ploc[0] + psize, ploc[1] : ploc[1] + psize,
                        ploc[2] : ploc[2] + psize] for ploc in patch_locations])

    dy = np.array([i_dy[ploc[0] : ploc[0] + psize, ploc[1] : ploc[1] + psize,
                        ploc[2] : ploc[2] + psize] for ploc in patch_locations])
    
    dz = np.array([i_dz[ploc[0] : ploc[0] + psize, ploc[1] : ploc[1] + psize,
                        ploc[2] : ploc[2] + psize] for ploc in patch_locations])

    dx = dx.reshape((ncells**3, psize**3))
    dy = dy.reshape((ncells**3, psize**3))
    dz = dz.reshape((ncells**3, psize**3))

    #collect all gradients in one array and calculate magnitudes
    raw_gradients = np.dstack((dx, dy, dz))
    if rot_inv is True:
        #rotate region according to dominant direction to achieve rotational invariance
        R = structure_tensor_3D(raw_gradients, getAverageMask(rsize, 'gaussian'))
        gradients = R.T.dot(raw_gradients.reshape((-1, 3)).T) 
        gradients = gradients.reshape(3, raw_gradients.shape[1], raw_gradients.shape[0]).T
    else:
        gradients = raw_gradients
    gradient_magnitudes = np.linalg.norm(gradients, axis = 2)

    #project gradients to icosahedron orientation axes
    projected_gradients = gradients.dot(axes.T)
    projected_gradients /= (gradient_magnitudes[:, :, None]+0.001)

    # compute theshold to clip projected gradients and recalculate magnitude
    inner_prods = linear_kernel(axes)[0, :]
    thres = np.sort(inner_prods)[-2]

    projected_gradients -= thres
    projected_gradients[projected_gradients < 0] = 0
    projected_gradient_magnitudes = np.linalg.norm(projected_gradients, axis = 2)

    #distribute original magnitude in orientation bins
    gradient_histograms = projected_gradients * (gradient_magnitudes[:, :, None] / (projected_gradient_magnitudes[:, :, None]+0.001))
    D = gradient_histograms.sum(axis = 1)

    if mode == 'flatten':
        Descriptor = (region_mask.ravel()[:, None] * D).ravel()
    else:
        Descriptor = region_mask.ravel().dot(D)

    if norm == 'l2':
        Descriptor = Descriptor / np.linalg.norm(Descriptor)
    if norm == 'l2-hys':
        Descriptor = Descriptor / np.linalg.norm(Descriptor)
        Descriptor = np.clip(Descriptor, a_min = 0, a_max = 0.25)
        Descriptor = Descriptor / np.linalg.norm(Descriptor)
    
    return Descriptor
   

def fn_hog_lbp_descriptor(coordinate, array):  
    descriptor_lbp = lbp_ri_sh(coordinate, array, 5, 4, 2, 2, concatenate = True)
    descriptor_hog = hog_3d_proj(coordinate, array) 
    descriptor_lbp_hog = np.concatenate([descriptor_hog,descriptor_lbp])
    return descriptor_lbp_hog.reshape(1, len(descriptor_lbp_hog))

# CODEWORDS

In [42]:
all_descriptors = []

In [43]:

def cw_coordinate_sampling(segm, roi):
    coord_labels = []
    coordinates = []
    
    mask_values = segm[roi_perimeter[:,0], roi_perimeter[:,1], roi_perimeter[:,2]]

    threshold_4 = np.count_nonzero(mask_values == 4) #.shape[0]#len(segm[segm[coordinate_cube]==4])
    
    threshold_1=int(threshold_4/1.3)
    threshold_2=int(threshold_4*1.3)
    threshold_3=int(threshold_4/1.3)
    threshold_0=int(threshold_4/1.4)
   
    i_0,i_1,i_2,i_3,i_4 =0, 0, 0, 0, 0
    
        
        
    for coordinate in roi_perimeter:
        x,y,z = coordinate
        if roi[x,y,z]==1:
            if segm[x,y,z]==0 and i_0<threshold_0:
                coordinates.append(coordinate)
                i_0+=1
                descriptor = fn_hog_lbp_descriptor(coordinate, mri_scan)
                descriptor = np.reshape(descriptor, [descriptor.shape[1],])
                all_descriptors.append(descriptor)    
            elif segm[x,y,z]==1 and i_1<threshold_1:
                coordinates.append(coordinate)
                i_1+=1
                descriptor = fn_hog_lbp_descriptor(coordinate, mri_scan)
                descriptor = np.reshape(descriptor, [descriptor.shape[1],])
                all_descriptors.append(descriptor)    
            elif segm[x,y,z]==2 and i_2<threshold_2:
                coordinates.append(coordinate)
                i_2+=1
                descriptor = fn_hog_lbp_descriptor(coordinate, mri_scan)
                descriptor = np.reshape(descriptor, [descriptor.shape[1],])
                all_descriptors.append(descriptor)    
            elif segm[x,y,z]==3 and i_3<threshold_3:
                coordinates.append(coordinate)
                i_3+=1
                descriptor = fn_hog_lbp_descriptor(coordinate, mri_scan)
                descriptor = np.reshape(descriptor, [descriptor.shape[1],])
                all_descriptors.append(descriptor)    
            elif segm[x,y,z]==4 and i_4<threshold_4:
                coordinates.append(coordinate)
                i_4+=1 
                descriptor = fn_hog_lbp_descriptor(coordinate, mri_scan)
                descriptor = np.reshape(descriptor, [descriptor.shape[1],])
                all_descriptors.append(descriptor)    
                

    # for value, count in zip(unique_values, counts):
    #     print(f"{value}: {count}")
    coordinates = np.array(coordinates)  
    np.random.shuffle(coordinates)      
    return coordinates

In [50]:
for scan_no in os.listdir('data'):
    try:
        print(scan_no)
        mri_scan = fn_scan_to_array('data/' + scan_no)
        segm_mask = fn_segm_mask_to_array(scan_no)
        roi = np.load('data/'+ scan_no+ '/roi.npy')
        coordinates = cw_coordinate_sampling(segm_mask, roi)
        print('got coordinates for scan no' , scan_no)
        np.save('all_descriptors.npy', all_descriptors)
    except Exception:
        pass

9001104
got coordinates for scan no 9001104
9002430
got coordinates for scan no 9002430
9002817
got coordinates for scan no 9002817
9003430
got coordinates for scan no 9003430
9004175
got coordinates for scan no 9004175
9005075
got coordinates for scan no 9005075
9005132
got coordinates for scan no 9005132
9006723
got coordinates for scan no 9006723
9007827
9008561
9011115
got coordinates for scan no 9011115
9011420
9012867
9013161
9017909
got coordinates for scan no 9017909
9019287
got coordinates for scan no 9019287
9023193
got coordinates for scan no 9023193
9027422
9031426
9033937
got coordinates for scan no 9033937
9034644
got coordinates for scan no 9034644
9036287
got coordinates for scan no 9036287
9036770
got coordinates for scan no 9036770
9036948
got coordinates for scan no 9036948
9039627
9040944
got coordinates for scan no 9040944
9041946
got coordinates for scan no 9041946
9044005
9047539
got coordinates for scan no 9047539
9052335
got coordinates for scan no 9052335
9052

In [51]:
k = len(os.listdir("data"))

print(np.load("all_descriptors.npy").shape[0] / k)

1013.7391304347826


In [52]:
def find_codewords(local_descriptors, no_of_words):
    random_state=42
    kmeans_model=KMeans(n_clusters=no_of_words, verbose=False, init='k-means++', random_state=random_state)
    kmeans_model.fit(local_descriptors)
    return kmeans_model.cluster_centers_

In [53]:
all_descriptors = np.array(all_descriptors)
np.save('all_descriptors.npy', all_descriptors)

In [54]:
all_descriptors[0] - all_descriptors[1]

array([ 0.00000000e+00,  1.91952094e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -3.04163061e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.95000876e-03,  0.00000000e+00,  0.00000000e+00,
        1.08101642e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -1.85031697e-02, -6.08567310e-03,  4.31336692e-03,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  7.19988154e-09,  0.00000000e+00,
        1.14819179e-02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        2.40337451e-02, -2.76542166e-08,  1.19314743e-03,  1.36857220e-02,
       -5.60597150e-02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -4.05641340e-08,  0.00000000e+00,  2.15605089e-02,  9.42096335e-02,
        1.39259416e-01,  2.59231188e-02, -1.66928178e-02, -1.10397370e-01,
       -2.51654970e-02, -1.85227800e-01, -7.21440434e-02, -2.95281069e-02,
        3.24670336e-02,  

In [55]:
codewords = find_codewords(all_descriptors, 40)
np.save('codewords.npy', codewords)

c:\MySoftware\python3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [57]:
codewords.shape

(40, 192)

In [15]:
import os 
import SimpleITK as sitk
import numpy as np
i,j=0,0
for scan_no in os.listdir('data'):
    try:
        mri_scan = fn_scan_to_array('data/' + scan_no)
        i+=1
    except Exception:
        j+=1
        
print(i,j)
        

28 18


In [ ]:
i= 0
AVAILABLE_SCANS = []

for scan_no in os.listdir()